# Search Engine

This lab is about starting from scratch, and hence having to structure the code yourself. A specification for the program you are to implement is given below - do pay attention as misunderstandings may cost you marks! Being precise is a necessary skill for a programmer. To give an executive summary, you are to code a search engine for recipes. A data set has been provided. The search engine is to be pretty basic, returning all recipes that contain all of the provided keywords. However, the user can choose from a number of orderings depending on their food preferences, which you need to support.

## Marking and submission

These lab exercises are marked, and contribute to your final grade. This lab exercise has 20 marks to earn, equivalent to 10% of your final grade.

Please submit your completed workbook to the auto marker before 8th November, 20:00 GMT. The workbook you submit must be an .ipynb file, which is saved into the directory you're running Jupyter; alternatively you can download it from the menu above using `File -> Download As -> Notebook (.ipynb)`. Remember to save your work regularly (`Save and checkpoint` in the `File` menu, the icon of a floppy disk, or `Ctrl-S`). It is wise to verify it runs to completion with _Restart & Run All_ before submission.

You must comply with the universities plagiarism guidelines: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

## Specification

The system must provide a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. Must `print` just their title, one per line. Must be less than `count` if the search is specific enough that less than `count` recipes match.

As an aside, do not worry about memory usage. If duplicating some data can make your code faster/neater then feel free.



### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. Your code will need to cope with this.

You will probably want to explore the data before starting, so you have an idea of what your code has to deal with.

Data set came from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though note it has been cleaned it up, by deleting duplicates and removing the really dodgy entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is to be ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the `re` module, but there is no need to do that here (`re` can be quite fiddly). For matching words your tokenisation must follow the following steps:
1. Convert all punctuation and digits into spaces. For punctuation use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits). You may find [`translate()`](https://docs.python.org/3/library/stdtypes.html#str.translate) interesting!
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search (above) or ordering (below) it's only a match if you match an entire token. There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The auto marker will be checking the above is followed! When doing a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$)

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet. Try not to overthink the optimisation of $n$, as it's really quite simple to do!

To clarify the use of the ordering string, to get something healthy that contains cheese you might call `search('cheese', 'healthy')`. In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry in the recipe (e.g. no `ingredients` are provided) to simply mean that entry can't match any search words (because it has none!), but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.



### Extra

You may find the [_inverted index_](https://en.wikipedia.org/wiki/Inverted_index) interesting. It's a data structure used by search engines. For each word a user may search for this contains a list of all documents (recipes) that contain the word. This may take a little effort to understand, but the resulting code will be both faster and neater.

## Advice

* Don't just start coding: Make a plan and work out what you intend to do.
* Think about structure, as messy code leads to mistakes.
* Plan your data structures. Don't be afraid to use sheets of paper if that works for you!
* Don't duplicate code, put it in a function/method instead.

* Divide and conquer. Break the system into parts that can implemented with minimal dependency on the rest. Functions or OOP are both suitable for doing this.
* Test early. Verify the individual parts work before trying to combine them. Factor this into the order you implement the parts of the system - don't implement something you are going to struggle to test before implementing, and verifying, other parts.
* Do not try and do a 'big bang', where you get it all working at once. Instead, get it working with features missing, then add those features in, one at a time.

* Keep things as simple as possible. Avoid long functions/methods.
* Include comments, as a form of planning and for your own sanity!
* Regularly reset the kernel and rerun the entire workbook. It is very easy to break something but not notice, because the correct version remains in memory.

## Marks
* __20 marks__: For many different _unit tests_ that will be run on `search`. They will cover all of the details in the above specification!
    * One test checks that it's faster than $0.1$ seconds on average (on the computer which runs the auto marker, which is quite fast) to do a search, so try not to be too inefficient (it ignores any time your notebook spends building data structures to be used by `search`). Note that the validation implementation comes in at $0.001$ seconds per search (after $5.5$ seconds of preparation), so this is very achievable!
    * You may want to look into Python's `set()` object, as it is useful for _one_ of the possible ways to implement this.
    * There will be sorting. The [Sorting how to](https://docs.python.org/3/howto/sorting.html) may help.
    * The auto marker does give some feedback, and you can run it as many times as you want. Don't be afraid to test an incomplete or semi-broken version of your code if stuck or unsure. It may help!
    * The validation implementation is 104 lines of code split over 5 cells (including white space for clarity and comments). Coded by someone who probably has much more experience than you, so you shouldn't aim to match this, but it's a good clue: If you find yourself at 500 lines you may want to stop and think some more! (line count does not include testing code, which is about the same amount again)

## Dataset Loading and Clean up

In [1]:
import json
import numpy
import pandas
import string

with open('recipes.json', 'r') as json_file:
    data = json.load(json_file)

# Loading the json file to a Pandas DataFrame
dataDF = pandas.DataFrame(data)

print(dataDF.shape)

(17734, 8)


In [2]:
# Mapping punctuations and strings to space values
# Reference
# https://stackoverflow.com/questions/12437667/how-do-i-replace-punctuation-in-a-string-in-python/12437738

remove_punc = str.maketrans(string.punctuation, ' '*len(string.punctuation))
remove_digits = str.maketrans(string.digits, ' '*len(string.digits))

## Tokenisation

In [3]:
def Tokenisation(dataDF):
    """Create a dictionary, which for every word it has the recipes it is present in
        Structure: {word:[title1, title2], word2:[title1,title3], ...}
    """
    
    tokensDict = {}

    for index in range(len(dataDF)): 
        
        uniqueWord = []
        
        # ------------------------------------------------------------
        
        title = dataDF.iloc[index]["title"]
        
        # Removing punctuation, digits and making it lowercase
        filteredTitle = title
        filteredTitle = filteredTitle.translate(remove_punc)
        filteredTitle = filteredTitle.translate(remove_digits).lower().replace('—',' ').split()
        
        Titlewords = numpy.unique(filteredTitle)
        
        for word in Titlewords:
            if len(word) > 2 and word not in uniqueWord:
                uniqueWord.append(word)
             
        # ------------------------------------------------------------
        
        categoryList = dataDF.iloc[index]["categories"]

        if isinstance(categoryList, list):
            
            # Converting a list to a string
            # Reference
            # https://www.geeksforgeeks.org/python-program-to-convert-a-list-to-string/
            category = ' '.join([str(word) for word in categoryList])

            filteredCategory = category
            filteredCategory = filteredCategory.translate(remove_punc)
            filteredCategory = filteredCategory.translate(remove_digits).lower().split() #.replace('—',' ')

            Categorywords = numpy.unique(filteredCategory)
               
            for word in Categorywords:
                if len(word) > 2 and word not in uniqueWord:
                    uniqueWord.append(word)      
        
        # ------------------------------------------------------------
        
        IngredientsList = dataDF.iloc[index]["ingredients"]
        
        if isinstance(IngredientsList, list):

            ingredients = ' '.join([str(word) for word in IngredientsList])

            filteredingredients = ingredients
            filteredingredients = filteredingredients.translate(remove_punc)
            filteredingredients = filteredingredients.translate(remove_digits).lower().split()

            Ingredientswords = numpy.unique(filteredingredients)

            for word in Ingredientswords:
                if len(word) > 2 and word not in uniqueWord:
                    uniqueWord.append(word)     
                    
        # ------------------------------------------------------------

        directionsList = dataDF.iloc[index]["directions"]
        
        if isinstance(directionsList, list):

            directions = ' '.join([str(word) for word in directionsList])

            filteredDirections = directions
            filteredDirections = filteredDirections.translate(remove_punc)
            filteredDirections = filteredDirections.translate(remove_digits).lower().split()

            Directionswords = numpy.unique(filteredDirections)
            
            for word in Directionswords:
                if len(word) > 2 and word not in uniqueWord:
                    uniqueWord.append(word)    
                
        # ------------------------------------------------------------

        for word in uniqueWord:
            if word not in tokensDict:
                tokensDict[word] = [title]
            elif title not in tokensDict[word]:
                tokensDict[word].append(title)
        
    return tokensDict

tokensDict = Tokenisation(dataDF)

## Ordering
Pre-processing for the Normal 'Ordering' Model

In [4]:
def normalPreProcessing(dataDF):
    """Pre-processing the ordering models helps tremendously with search speed
        For the Normal model I find every unique for all 4 parts of the recipe and calculate the words score using
        the given formula in the instructions. Its structure is in the form
        {title:{word:score, word:score...}, title:{word:score,...}, ...}
    """
    
    normalOrderScoreDict = {}
    ratingDict = {}
            
    for index in range(len(dataDF)): 
        
        indexUniqueWords = {}
        # ----------------------------- Normal Ordering -----------------------------
        
        title = dataDF.iloc[index]["title"]
        
        filteredTitle = title
        filteredTitle = filteredTitle.translate(remove_punc)
        filteredTitle = filteredTitle.translate(remove_digits).lower().split()
        
        # Finding the unique words with their count
        Titlewords, Titlecount = numpy.unique(filteredTitle, return_counts=True)            
        
        for i in range(len(Titlewords)):
            # Removing tokens that are less than 3 characters long
            if len(Titlewords[i]) > 2:
        
                if Titlewords[i] not in indexUniqueWords:
                    # Multiplying the count by the amount given in the formula for the specific part
                    indexUniqueWords[Titlewords[i]] = Titlecount[i] * 8 
                else:
                    indexUniqueWords[Titlewords[i]] += Titlecount[i] * 8
        
        # -----------------------------------------------------------------------------
        
        categoryList = dataDF.iloc[index]["categories"]
        
        # Check if the category is list and not a not Nan
        if isinstance(categoryList, list):
        
            category = ' '.join([str(word) for word in categoryList])

            filteredCategory = category
            filteredCategory = filteredCategory.translate(remove_punc)
            filteredCategory = filteredCategory.translate(remove_digits).lower().split()

            Categorywords, Categorycount = numpy.unique(filteredCategory, return_counts=True)

            for i in range(len(Categorywords)):
                if len(Categorywords[i]) > 2:
                    if Categorywords[i] not in indexUniqueWords:
                        indexUniqueWords[Categorywords[i]] = Categorycount[i] * 4
                    else:
                        indexUniqueWords[Categorywords[i]] += Categorycount[i] * 4
        
        # -----------------------------------------------------------------------------
        
        IngredientsList = dataDF.iloc[index]["ingredients"]
        
        if isinstance(IngredientsList, list):
        
            ingredients = ' '.join([str(word) for word in IngredientsList])

            filteredingredients = ingredients
            filteredingredients = filteredingredients.translate(remove_punc)
            filteredingredients = filteredingredients.translate(remove_digits).lower().split()

            Ingredientswords, Ingredientscount = numpy.unique(filteredingredients, return_counts=True)
                
            for i in range(len(Ingredientswords)):
                if len(Ingredientswords[i]) > 2:
                    if Ingredientswords[i] not in indexUniqueWords:
                        indexUniqueWords[Ingredientswords[i]] = Ingredientscount[i] * 2
                    else:
                        indexUniqueWords[Ingredientswords[i]] += Ingredientscount[i] * 2
            
        # -----------------------------------------------------------------------------

        directionsList = dataDF.iloc[index]["directions"]

        if isinstance(directionsList, list):

            directions = ' '.join([str(word) for word in directionsList])

            # Removing punctuation, digits and making it lowercase
            filteredDirections = directions
            filteredDirections = filteredDirections.translate(remove_punc)
            filteredDirections = filteredDirections.translate(remove_digits).lower().split()

            Directionswords, Directionscount = numpy.unique(filteredDirections, return_counts=True)
                            
            for i in range(len(Directionswords)):
                # Removing tokens that are less than 3 characters long
                if len(Directionswords[i]) > 2:
                    if Directionswords[i] not in indexUniqueWords:
                        indexUniqueWords[Directionswords[i]] = Directionscount[i]
                    else:
                        indexUniqueWords[Directionswords[i]] += Directionscount[i]

        # -----------------------------------------------------------------------------
        
        rating = dataDF.iloc[index]["rating"]
        
        if numpy.isnan(rating):
            rating = 0
                
        ratingDict[title] = rating
        normalOrderScoreDict[title] = indexUniqueWords
        
    return normalOrderScoreDict, ratingDict


normalOrderScoreDict, ratingDict = normalPreProcessing(dataDF)

Pre-processing for the 'Simple' and 'Healthy' ordering models

In [5]:
def simpleHealthyPreProcessing(dataDF):
    """For every recipe I calculate its simple and healthy score and add them to a dictionary to a structure
    {title:score, title:Score, ....}"""

    simpleOrderScoreDict = {}
    healthyOrderScoreDict = {}
                           
    for index in range(len(dataDF)): 
        
        title = dataDF.iloc[index]["title"]

        IngredientsList = dataDF.iloc[index]["ingredients"]
        directionsList = dataDF.iloc[index]["directions"]
        
        # ----------------------------- Simple Ordering -----------------------------
        
        if not isinstance(IngredientsList, list) or not isinstance(directionsList, list):
            simpleOrderScoreDict[title] = numpy.nan
        else:
            
            numIngredients = len(IngredientsList)
            numDirections = len(directionsList)

            if numIngredients == 1 or numDirections == 1:
                simpleOrderScoreDict[title] = numpy.nan
            else:
                simpleOrderScoreDict[title] = (numIngredients * numDirections)
                
        # ----------------------------- Healthy Ordering -----------------------------

        calories = dataDF.iloc[index]['calories']
        protein = dataDF.iloc[index]['protein']
        fat = dataDF.iloc[index]['fat']
                
        # Get the initial Cost where n=1
        cost = (abs(calories - 510*1)/510) + (2* abs(protein - 18*1)/18) + (4* abs(fat - 150*1)/150)

        for n in range(2,6):
            newcost = (abs(calories - 510*n)/510) + (2* abs(protein - 18*n)/18) + (4* abs(fat - 150*n)/150)
        
            # If the cost with the new n is lower it replaces the old one
            if newcost < cost:
                cost = newcost
        
        healthyOrderScoreDict[title] = cost

    return simpleOrderScoreDict, healthyOrderScoreDict

simpleOrderScoreDict, healthyOrderScoreDict = simpleHealthyPreProcessing(dataDF)

Loading the ordering model functions

In [6]:
def order_normal(queryList, recipeList):
    """normal - Based simply on the number of times the search terms appear in the recipe. 
    A score is calculated and the order is highest to lowest. 
    8 x Number of times a query word appears in the title
    4 x Number of times a query word appears in the categories
    2 x Number of times a query word appears in the ingredients
    1 x Number of times a query word appears in the directions
    + The rating of the recipe 
    """
    
    recipeListScoreDict = {}
    
    for title in recipeList:
                
        rating = ratingDict[title]  
        
        # Add rating for the specific recipe
        score = rating
        
        for query in queryList:
            score += normalOrderScoreDict[title][query]
        
        recipeListScoreDict[title] = score
        
    # Sorting by score and to an ordered list of titles
    # Reference
    # https://stackabuse.com/how-to-sort-dictionary-by-value-in-python/
    recipeListSorted = sorted(recipeListScoreDict, key=recipeListScoreDict.get, reverse=True)
        
    return recipeListSorted
    

def order_simple(recipeList):
    """simple - Tries to minimise the complexity of the recipe, for someone who is in a rush. 
    Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.
    """
    
    # Create simpleOrderScore column
    # For every recipe it has the number of ingredients * number of steps in the directions
    
    recipeListScoreDict = {}
    
    for title in recipeList:
            
        simpleScore = simpleOrderScoreDict[title]

        # Drops the recipe if the value is NaN
        if not numpy.isnan(simpleScore):
            recipeListScoreDict[title] = simpleScore
                
    # Sorting by lowest score and to an ordered list of titles
    recipeListSorted = sorted(recipeListScoreDict, key=recipeListScoreDict.get)
    
    return recipeListSorted


def order_healthy(recipeList):
    """healthy - Order from lowest to highest by this cost function
     This can be understood in terms of the numbers for calories, protein and fat, being a third of the recommended 
     daily intake (three meals per day) for an average person. So this tries to select recipes that neatly divide 
     into a set of meals that are the right amount to consume for a healthy, balanced diet.
    """
    
    # Order from lowest to highest
    recipeListScoreDict = {}
    
    for title in recipeList:
        
        healthyScore = healthyOrderScoreDict[title]
        
        # Drops the recipe if the value is NaN
        if not numpy.isnan(healthyScore):
            recipeListScoreDict[title] = healthyScore
            
    # Sorting by lowest score and to an ordered list of titles
    recipeListSorted = sorted(recipeListScoreDict, key=recipeListScoreDict.get)
    
    return recipeListSorted

## Search

In [12]:
def search(query, ordering = 'normal', count = 10):
    """ The search function prints the recipes that contain all of the words in the query. 
        In the parts of Title, Category, Ingredients, Directions. It then orders them according to the model provided
        and returns them using the count.
    """
    
    # For puctuation and uppercase
    filteredQuery = query
    filteredQuery = filteredQuery.translate(remove_punc)
    queryList = filteredQuery.translate(remove_digits).lower().split()
    
    # For testing short tokens
    for query in queryList:
        if len(query) < 3 :
            queryList.remove(query)
    
    # Checks if query is now empty
    if len(queryList) == 0:
        return
    
    # Checks for no matches for the query given
    try:
        tokensDict[queryList[0]]
        recipeList = tokensDict[queryList[0]]
    except:
        return
    
    # Checks for no matches for the query given
    for query in queryList:
        try:
            tokensDict[query]
            resultList = tokensDict[query]
        except:
            return
        
        recipeList = list(set(recipeList) & set(resultList))

    # For No Matches
    if len(recipeList) == 0:
        return
    
    # Order the recipes that match the query based on the chosen model
    if ordering == 'normal':
        recipeListSorted = order_normal(queryList, recipeList)
                
    elif ordering == 'simple':
        recipeListSorted = order_simple(recipeList)
        
    elif ordering == 'healthy':
        recipeListSorted = order_healthy(recipeList)
    
    # Returns if the ordering model chosen is invalid
    else:
        return

    # Print the sorted list according to count
    # If the list of recipes is less than the count it should print all of them
    # For Few Matches
    if len(recipeListSorted) < count:
            for recipe in recipeListSorted:
                print(recipe)
    else:
        for recipe in recipeListSorted[0:count]:
            print(recipe)


In [8]:
search('peanut butter', 'normal', 7)

Chocolate Cake with Milk Chocolate-Peanut Butter Frosting and Peanut Butter Brittle 
Peanut Butter Tart with Caramel-Peanut Glaze 
Peanut Butter Cake with Chocolate-Peanut Butter Icing 
Peanut Butter Banana Cream Pie 
Banana Cupcakes with Peanut Butter Frosting 
Chocolate-Peanut Butter Cake with Cream Cheese and Butterfinger Frosting 
Peanut Butter Cheesecake with Peanut Brittle 


In [16]:
search('chocolate ds', 'simple', 2)

Chocolate Leaves 
Raw Nut Butter 


In [26]:
search('baked potato', 'simple', 3)

Tiny Baked Potatoes with Chervil Cream 
Baked Whole Fish with Potatoes and Lemon 
Twice-Baked Potatoes with Basil and Sour Cream 


In [27]:
search('tiramisu', 'healthy')

Mixed Berry Tiramisu 
Chocolate Swirl Tiramisu 
Tiramisu Ice-Cream Cake 
Super-Simple Pumpkin Tiramisu 
Strawberry Tiramisu 
Caramelized Fresh Pineapple Tiramisu 
Amaretti Tiramisu 
White Chocolate Tiramisu Trifle With Spiced Pears 
Tiramisu Affogato 
Berry Tiramisu 


In [28]:
search('pie', 'healthy', 20)

Corn and Bacon Pie 
Spinach, Red Pepper, and Feta Quiche 
Zucchini, Sun-Dried Tomato, and Mozarella Tart 
Flaky Pie Crust Dough 
Chocolate Mousse Pie 
Bacon and Green Chili Quiche 
Yellow Squash and Mozzarella Quiche with Fresh Thyme 
Scallop and Shrimp Fritters with Chipotle Mayonnaise 
Butter Pie Crust 
Challah, Sausage, and Dried Cherry Stuffing 
Frozen Peanut Butter Pie with Candied Bacon 
Butter Pie Crust Dough 
Warm Goat Cheese Salad with Pears and Walnuts 
Kale, Butternut Squash, and Pancetta Pie 
Asparagus Tart 
Flaky Pie Crust 
Caramelized Garlic, Spinach, and Cheddar Tart 
Pear and Almond Tart 
Bacon and Potato Pie 
Sweet Cheese Pie 
